# Word and POS tags embeddings


## Description
In this notebook, we are trying to parse $*.conllu$ files and extract the word embeddings, as well as the POS tags emeddings.

## Work to be done:
1. Learn how to parse $*.conllu$ files
2. Find a way to get all words and POS tags embeddings.
3. Prepare data to be ready to be used by BiLSTM

### 1.1 Read $*.conllu$ files

#### Word class:

In [23]:
class Word:
    """Word class that maps all details for a word from a *.conllu file. """
    
    def __init__(self, ID, FORM, LEMMA, UPOSTAG, XPOSTAG, FEATS, HEAD, DEPREL, DEPS, MISC):
        self.ID = ID           # Word idx, int starting at 1 for each new sentence; or a range for multiword tokens
        self.FORM = FORM       # Word form or punctuation symbol.
        self.LEMMA  = LEMMA    # Lemma or stem of word form.
        self.UPOSTAG = UPOSTAG # Universal part-of-speech tag.
        self.XPOSTAG = XPOSTAG # Language-specific part-of-speech tag; underscore if not available.
        self.FEATS = FEATS     # List of morphological features from the universal feature inventory
        self.HEAD = HEAD       # Head of the current word, which is either a value of ID or zero (0).
        self.DEPREL = DEPREL   # Univ. dep. rel to the HEAD (root iff HEAD = 0) or a defined lang-specific subtype
        self.DEPS = DEPS       # Enhanced dependency graph in the form of a list of head-deprel pairs.
        self.MISC = MISC       # Any other annotation.
    
    def __str__(self):
        string = ''
        
        string += self.ID      if self.ID      != None else '_'
        string += " "
        string += self.FORM    if self.FORM    != None else '_'
        string += " "
        string += self.LEMMA   if self.LEMMA   != None else '_'
        string += " "
        string += self.UPOSTAG if self.UPOSTAG != None else '_'
        string += " "
        string += self.XPOSTAG if self.XPOSTAG != None else '_'
        string += " "
        string += self.FEATS   if self.FEATS   != None else '_'
        string += " "
        string += self.HEAD    if self.HEAD    != None else '_'
        string += " "
        string += self.DEPREL  if self.DEPREL  != None else '_'
        string += " "
        string += self.DEPS    if self.DEPS    != None else '_'
        string += " "
        string += self.MISC    if self.MISC    != None else '_'
        
        return string
        
    @staticmethod
    def from_line(line):
        tokens = line.split()
        
        assert len(tokens) == 10
        
        ID =      tokens[0] if tokens[0] != '_' else None
        FORM =    tokens[1] if tokens[1] != '_' else None
        LEMMA =   tokens[2] if tokens[2] != '_' else None
        UPOSTAG = tokens[3] if tokens[3] != '_' else None
        XPOSTAG = tokens[4] if tokens[4] != '_' else None
        FEATS =   tokens[5] if tokens[5] != '_' else None
        HEAD =    tokens[6] if tokens[6] != '_' else None
        DEPREL=   tokens[7] if tokens[7] != '_' else None
        DEPS =    tokens[8] if tokens[8] != '_' else None
        MISC =    tokens[9] if tokens[9] != '_' else None
        
        return Word(ID, FORM, LEMMA, UPOSTAG, XPOSTAG, FEATS, HEAD, DEPREL, DEPS, MISC)
        

#### Test Word class:

In [24]:
test_text = "4	proves	prove	VERB	VBZ	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	0	root	0:root	_"
test_word = Word.from_line(test_text)
print(test_word)
print(test_word.FORM)

4 proves prove VERB VBZ Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin 0 root 0:root _
proves


#### Sentence class:

In [25]:
class Sentence:
    """Sentence class that maps all details for a sentence from a *.conllu file. """
    
    def __init__(self,newdoc_id, send_id, text, words):
        self.newdoc_id = newdoc_id
        self.send_id = send_id
        self.text = text
        self.words = words
        
    def __str__(self):
        string = ''
        if self.newdoc_id != None:
            string += "# newdoc id = "
            string += str(self.newdoc_id)
            string += "\n"
        string += "# send_id = "
        string += str(self.send_id)
        string += "\n"
        string += "# text = "
        string += str(self.text)
        string += "\n"
        sentence_length = len(self.words)
        for index, word in enumerate(self.words):
            string += str(word)
            if index < sentence_length - 1:
                string += "\n"
        return string
    
        
    @staticmethod
    def from_lines(lines):
        newdoc_id=''
        send_id = ''
        text=''
        words = []

        
        for line in lines:
            if line.startswith("#"): #misc properties
                prop = line.split("=")[1]
                if line.startswith("newdoc", 2):
                    newdoc_id = prop
                    continue
                if line.startswith("sent_id", 2):
                    send_id = prop
                    continue
                if line.startswith("text", 2):
                    text = prop
                    continue
                continue
            else:# words
                if line.split()[0].isdigit(): #index is an integer
                    words.append(Word.from_line(line))

        return Sentence(newdoc_id, send_id, text, words)

#### Test Sentence class:

In [26]:
lines = ["# sent_id = weblog-blogspot.com_gettingpolitical_20030906235000_ENG_20030906_235000-0003",
"# text = Today's incident proves that Sharon has lost his patience and his hope in peace.",
"1	Today	today	NOUN	NN	Number=Sing	3	nmod:poss	3:nmod:poss	SpaceAfter=No",
"2	's	's	PART	POS	_	1	case	1:case	_",
"3	incident	incident	NOUN	NN	Number=Sing	4	nsubj	4:nsubj	_",
"4	proves	prove	VERB	VBZ	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	0	root	0:root	_",
"5	that	that	SCONJ	IN	_	8	mark	8:mark	_",
"6	Sharon	Sharon	PROPN	NNP	Number=Sing	8	nsubj	8:nsubj	_",
"7	has	have	AUX	VBZ	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	8	aux	8:aux	_",
"8	lost	lose	VERB	VBN	Tense=Past|VerbForm=Part	4	ccomp	4:ccomp	_",
"9	his	he	PRON	PRP$	Gender=Masc|Number=Sing|Person=3|Poss=Yes|PronType=Prs	10	nmod:poss	10:nmod:poss	_",
"10	patience	patience	NOUN	NN	Number=Sing	8	obj	8:obj	_",
"11	and	and	CCONJ	CC	_	13	cc	13:cc	_",
"12	his	he	PRON	PRP$	Gender=Masc|Number=Sing|Person=3|Poss=Yes|PronType=Prs	13	nmod:poss	13:nmod:poss	_",
"13	hope	hope	NOUN	NN	Number=Sing	10	conj	10:conj	_",
"14	in	in	ADP	IN	_	15	case	15:case	_",
"15	peace	peace	NOUN	NN	Number=Sing	13	nmod	13:nmod	SpaceAfter=No",
"16	.	.	PUNCT	.	_	4	punct	4:punct	_"]
sentence = Sentence.from_lines(lines)
print(str(sentence), "\n")
print(sentence.text, "\n")
print(sentence.words[0], "\n")

# newdoc id = 
# send_id =  weblog-blogspot.com_gettingpolitical_20030906235000_ENG_20030906_235000-0003
# text =  Today's incident proves that Sharon has lost his patience and his hope in peace.
1 Today today NOUN NN Number=Sing 3 nmod:poss 3:nmod:poss SpaceAfter=No
2 's 's PART POS _ 1 case 1:case _
3 incident incident NOUN NN Number=Sing 4 nsubj 4:nsubj _
4 proves prove VERB VBZ Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin 0 root 0:root _
5 that that SCONJ IN _ 8 mark 8:mark _
6 Sharon Sharon PROPN NNP Number=Sing 8 nsubj 8:nsubj _
7 has have AUX VBZ Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin 8 aux 8:aux _
8 lost lose VERB VBN Tense=Past|VerbForm=Part 4 ccomp 4:ccomp _
9 his he PRON PRP$ Gender=Masc|Number=Sing|Person=3|Poss=Yes|PronType=Prs 10 nmod:poss 10:nmod:poss _
10 patience patience NOUN NN Number=Sing 8 obj 8:obj _
11 and and CCONJ CC _ 13 cc 13:cc _
12 his he PRON PRP$ Gender=Masc|Number=Sing|Person=3|Poss=Yes|PronType=Prs 13 nmod:poss 13:nmod:poss _
13 

In [27]:
en_path = "Annotated training data/UD_English-master/"
en_dev_filename = "en-ud-dev.conllu"
en_test_filename = "en-ud-test.conllu"
en_train_filename = "en-ud-train.conllu"

ro_path = "Annotated training data/UD_Romanian-dev/"
ro_dev_filename = "ro-ud-dev.conllu"
ro_test_filename = "ro-ud-test.conllu"
ro_train_filename = "ro-ud-train.conllu"

def read_sentences(path, filename):
    sentences = []
    sentence_lines = []
    file = open(path + filename, 'r')
    for line in file:
        if line == '\n':
            sentences.append(Sentence.from_lines(sentence_lines))
            sentence_lines = []
        else:
            sentence_lines.append(line)
            
    return sentences


en_train_sentences = read_sentences(en_path,en_train_filename)
en_dev_sentences = read_sentences(en_path,en_dev_filename)
en_test_sentences = read_sentences(en_path,en_test_filename)

ro_train_sentences = read_sentences(ro_path,ro_train_filename)
ro_dev_sentences = read_sentences(ro_path,ro_dev_filename)
ro_test_sentences = read_sentences(ro_path,ro_test_filename)

assert len(en_dev_sentences) == 2002# there are 2002 sentences in the english development dataset.


### 1.2 Write $*.conllu$ files

In [28]:
def write_sentences(sentences, path, filename):
    with open(path + filename, 'w+') as file:
        for sentence in sentences:
            file.write(str(sentence) + "\n")

### 2. Extract words/POS tags and then extract embeddings

#### Function for displaying embedded words using t-SNE:

In [29]:
#### Imports
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE

from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.palettes import d3
from bokeh.io import output_notebook
output_notebook()

def emb_scatter(data, names, N=20, perplexity=30.0):
    """
    Function for plotting embeddings and words using TSNE.
    TSNE finds a way to plot multidimensional data to a 
    bidimensional plane. It assures that data close in the 
    multidimensionalspace will also be close in budimensional 
    place, but not the other way around.
    """
    ## try to find some clusters ##
    print("finding clusters")
    kmeans = KMeans(n_clusters=N)
    kmeans.fit(data)
    klabels = kmeans.labels_

    ## get a tsne fit ##
    print("fitting tsne")
    tsne = TSNE(n_components=2, perplexity=perplexity)
    emb_tsne = tsne.fit_transform(data)
    
    ## plot the tsne of the embeddings with bokeh ##
    # source: https://github.com/oxford-cs-deepnlp-2017/practical-1
    p = figure(tools="pan,wheel_zoom,reset,save",
               toolbar_location="above",
               title="T-SNE for most common words")

    # set colormap as a list
    colormap = d3['Category20'][N]
    colors = [colormap[i] for i in klabels]

    source = ColumnDataSource(data=dict(x1=emb_tsne[:,0],
                                        x2=emb_tsne[:,1],
                                        names=names,
                                        colors=colors))

    p.scatter(x="x1", y="x2", size=8, source=source, color='colors')

    labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                      text_font_size="8pt", text_color="#555555",
                      source=source, text_align='center')
    p.add_layout(labels)

    show(p)

Loading BokehJS ...

In [30]:
import numpy as np
import pickle
from collections import defaultdict, Counter
from random import random
import matplotlib.pyplot as plt
from gensim.models import Word2Vec

In [31]:
def get_Gensim_sentences(sentences):
    """
    This function receives a list of Sencences and return 
    the word and POS sentences in the format Gensim needs, 
    as well as Counters for words, POS tags and dependency
    relation labels.
    Format: [['i', 'like', 'custard'],...]
    All words are lower-cased
    """
    gensim_word_sentences = []
    word_counts = Counter()
    gensim_POS_sentences = []
    POS_counts = Counter()
    label_counts = Counter()
    for sentence in sentences:
        gensim_word_sentence = []
        gensim_POS_sentence = []
        for word in sentence.words:
            gensim_word_sentence.append(word.FORM.lower())
            word_counts[word.FORM.lower()] += 1
            gensim_POS_sentence.append(word.UPOSTAG.lower())
            POS_counts[word.UPOSTAG.lower()] += 1
            # some labels comprize multiple words(i.e: "nsubj:poss"), so we take only the first label
            label_counts[word.DEPREL.split(":")[0].lower()] += 1      
        gensim_word_sentences.append(gensim_word_sentence)
        gensim_POS_sentences.append(gensim_POS_sentence)
    return gensim_word_sentences, word_counts, gensim_POS_sentences, POS_counts, label_counts

In [32]:
gensim_word_sentences_train, word_counts_train, gensim_POS_sentences_train, POS_counts_train, label_counts_train = get_Gensim_sentences(en_test_sentences)
gensim_word_sentences_dev, word_counts_dev, gensim_POS_sentences_dev, POS_counts_dev, label_counts_dev = get_Gensim_sentences(en_dev_sentences)
gensim_word_sentences_test, word_counts_test, gensim_POS_sentences_test, POS_counts_test, label_counts_test = get_Gensim_sentences(en_test_sentences)


w2i = defaultdict(lambda: len(w2i))
i2w = dict()
i2w[w2i["<UNK>"]] = "<UNK>" # word with index 0 are the words that are unknown.
for word in word_counts_train.keys():
    if word_counts_train[word] > 1:
        i2w[w2i[word]] = word # trick
    else:
        w2i[word] = 0  # all unknown words have index 0
word_vocabulary = word_counts_train.keys()


        
t2i = defaultdict(lambda: len(t2i))
i2t = dict()
for tag in POS_counts_train.keys():
    i2t[t2i[tag]] = tag # trick
pos_tag_vocabulary = POS_counts_train.keys() 


        
l2i = defaultdict(lambda: len(l2i))
i2l = dict()
for label in label_counts_train.keys():
        i2l[l2i[label]] = label # trick



### a. Gensim Word2Vector trained on our data:

In [33]:
# gensim_word_model = Word2Vec([[i2w[w2i[word]] for word in sentence] for sentence in gensim_word_sentences_train], size=50, window=5, min_count=1, workers=4)
# gensim_POS_model = Word2Vec(gensim_POS_sentences_train, size=50, window=3, min_count=2, workers=4)

In [34]:
# top_words = [k for k,v in word_counts_train.most_common(1000)]
# top_word_vecs = gensim_word_model[top_words]
# emb_scatter(top_word_vecs, top_words, N=20)

In [35]:
# top_POS = [k for k,v in POS_counts_train.most_common(1000)]
# top_POS_vecs = gensim_POS_model[top_POS]
# emb_scatter(top_POS_vecs, top_POS, N=17)

### b. Gensim Word2Vector trained on Wikipedia data:

In [36]:
# TODO: implement or delete this section.

### c. GloVe word embeddings from  spacy library(300d):

In [37]:
# import spacy
# glove = spacy.load('en')

# def get_spacy_word_model():
#     spacy_word_model = {}
#     unknown_representation = []
#     for word in word_vocabulary:
#         if w2i[word] == 0:
#             unknown_representation.append(glove(word).vector)
#         else:
#             spacy_word_model[word] = glove(word).vector
#     return defaultdict(lambda: np.sum(unknown_representation, axis=0)/len(unknown_representation), spacy_word_model)

# spacy_word_model = get_spacy_word_model()

# def get_spacy_pos_model():
    
#     spacy_POS_model = {}
#     for POS_tag in pos_tag_vocabulary:
#         spacy_POS_model[POS_tag] = glove(POS_tag).vector
#     return spacy_POS_model

# spacy_POS_model=get_spacy_pos_model()

In [38]:
# most_common_words = word_counts_train.most_common(1000)
# most_common_words_vectors = []
# for word, count in most_common_words:
#     most_common_words_vectors.append(spacy_word_model[word])

# emb_scatter(most_common_words_vectors, [word for word, count in most_common_words], N=20)

In [39]:
# pos_vectors = []
# for pos_tag in pos_tag_vocabulary:
#     pos_vectors.append(spacy_POS_model[pos_tag])
# emb_scatter(pos_vectors, list(pos_tag_vocabulary), N=17)

### d. GloVe word embeddings from pre-trained word vectors(50d/100d):

In [74]:
# def get_glove_pre_trained_word_model():
#     # you need to download the file(s) from https://github.com/stanfordnlp/GloVe
#     file = open("GloveEmbeddings/glove.6B.50d.txt")
#     pre_trained_tokens = {}
#     for line in file:
#         tokens = line.split()
#         pre_trained_tokens[tokens[0]] = tokens[1:]

#     # ~2005 found words appearing >1, ~2500 found words appearing =1, ~400 words not found
#     pre_trained_unknown_reprez = []
#     pre_trained_dict = {}

#     for word in w2i.keys():
#         if word.lower() in pre_trained_tokens.keys():
#             pre_trained_dict[word] = pre_trained_tokens[word]
#             if w2i[word] == 0:
#                 pre_trained_unknown_reprez.append(pre_trained_tokens[word])
        
#     unk_reprez = np.sum(np.array(pre_trained_unknown_reprez).astype(np.float),axis=0)/len(pre_trained_unknown_reprez)

#     return defaultdict(lambda: unk_reprez, pre_trained_dict)

def get_glove_pre_trained_word_model():
    file = open("glove50d.txt")
    pre_trained_tokens = {}
    for line in file:
        tokens = line.split()
        if tokens[0] != "<UNK>":
            pre_trained_tokens[tokens[0]] = tokens[1:]
        else:
            unknown_reprez = tokens[1:]
    return defaultdict(lambda:unknown_reprez, pre_trained_tokens)


def get_glove_pre_trained_tag_model():
    """
    PLOT TWIST: we get the pos tag embeddings from training 
    """
    model = {}
    word2vect = Word2Vec(gensim_POS_sentences_train, size=50, window=3, min_count=2, workers=4)
    for key in word2vect.wv.vocab.keys():
        model[key] = word2vect.wv[key]
    return model
    
    
pre_trained_word_model = get_glove_pre_trained_word_model()
pre_trained_POS_model = get_glove_pre_trained_tag_model()

In [79]:
len(pre_trained_word_model.keys())

4549

In [78]:
word_vector = []
word_list = []
for word, count in word_counts_train.most_common(1000):
    word_list.append(word)
    word_vector.append(pre_trained_word_model[word])
emb_scatter(word_vector, word_list, N=20)

finding clusters
fitting tsne


In [42]:
top_POS = [k for k,v in POS_counts_train.most_common(1000)]

top_POS_vecs = []
for pos in top_POS:
    top_POS_vecs.append(pre_trained_POS_model[pos])
emb_scatter(top_POS_vecs, top_POS, N=17)

finding clusters
fitting tsne


### e. GloVe word embeddings trained from our corpus:

In [43]:
# Copy the "text8"  into the GloVe folder and run "./demo.sh". 
# Copy the "vector.txt" back to the notebook folder and run
# the second part of this cell.



# with open('text8', 'w+') as file:
#     for sentence in gensim_word_sentences_train:
#         for word in sentence:
#             if w2i[word] == 0:
#                 file.write("<UNK> ")
#             else:
#                 file.write(word + " ")
    
    
    
    
    
    

# trained_glove_word_model = {}

# with open('vectors.txt', 'r') as gloVe_corpus_file:
#     for line in gloVe_corpus_file:
#         l = line.split()
#         trained_glove_word_model[l.pop(0)] = l
        
        
# most_common_words = word_counts_train.most_common(1000)
# trained_word_vectors = []
# most_common_words = [ word for word, count in most_common_words]
# for word in most_common_words:
#     trained_word_vectors.append(trained_glove_word_model[i2w[w2i[word]]])
# emb_scatter(trained_word_vectors, most_common_words, N=20)

### 3. Prepare data to be used by BiLSTM

In [44]:
def get_word_embeddings(word_model, i2t):
    embeddings = []
    for index in i2w.keys():
        embeddings.append(word_model[i2w[index]])
    return embeddings

def get_tags_embeddings(tag_model, i2t):
    embeddings = []
    for index in i2t.keys():
        embeddings.append(tag_model[i2t[index]])
    return embeddings